# How to use Equity prices to estimate default probabilities?


In 1974, Merton published "On the Pricing of Corporate Debt: The Risk Structure of Interest Rates", in which he proposes a model where a company’s equity is an option on the assets of the company


$V_0$ : Value of company’s assets today

$V_T$ : Value of company’s assets at time T

$E_0$: Value of company’s equity today

$E_T$: Value of company’s equity at time T

$D$: Debt repayment due at time T

$\sigma_V$ : Volatility of assets (assumed constant)

$\sigma_E$ : Instantaneous volatility of equity

If $V_T$ < $D$, it is rational for the company to default on the debt at time $T$.The value of the equity $E_T = 0$

If $V_T$ > $D$, the company should make the debt repayment at time $T$ and the value of the equity $E_T = V_T- D$

Therefore, the value of the firm’s equity at time T is $E_T = max(V_T-D; 0)$

#### The Black–Scholes–Merton formula gives the value of the equity today as
#### (1) $$E_0 = V_0 N(d_1)-De^{rT}N(d_2)$$

$$d_1 = \frac{ln\left(V_0/D\right)+\left(r+\sigma_V^2/2\right)T}{\sigma_V\sqrt{T}}$$

$$d_2 = d_1 - \sigma_V \sqrt{T}$$

#### The risk-neutral probability that the company will default on the debt is 
$$1-N(d_2) = N(-d_2)$$\
To calculate this, we require $V_0$ and $\sigma_V$ We only have one equation for two unknowns, we need a second equation to estimate $V_0$ and $\sigma_V$ 

#### From Ito’s lemma we have, 
#### (2) $$\sigma_E E_0 = \frac{\partial{E}}{\partial{V}}\sigma_V V_0 = N(d_1)\sigma_V V_0$$

In [1]:
import numpy as np
import math as m
import scipy
import scipy.stats as si
from scipy import optimize

In [2]:
# First Equation, the B&S call price

def BS_call(V0, D, T, r, sigma_asset):
    
    #V0: value of company's asset today
    #D: debt repayment due at time T
    #T: time to maturity
    #r: interest rate
    #sigma_asset: volatility of assets, which is assumed constant
    
    d1 = (np.log(V0 / D) + (r + 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))
    d2 = (np.log(V0 / D) + (r - 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))
    
    call = (V0 * si.norm.cdf(d1, 0.0, 1.0) - D * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    # cdf means Cumulative distribution function
    
    return call

In [3]:
#Second Equation, from Ito's Lemma

def volatility_of_equity(V0, D, T, r, sigma_asset, E0):
    
    #V0: value of company's asset today
    #D: debt repayment due at time T
    #T: time to maturity
    #r: interest rate
    #sigma_asset: volatility of assets, which is assumed constant
    #E0: value of company's equity today
    
    d1 = (np.log(V0 / D) + (r + 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))
    
    volatility = si.norm.cdf(d1, 0.0, 1.0)*sigma_asset*V0*(1/E0)
    # cdf means Cumulative distribution function
    
    return volatility 

In [4]:
BS_call(12_400_000, 10_000_000, 1, 0.05, 0.2123)

3004198.184797439

In [5]:
volatility_of_equity(12_400_000, 10_000_000, 1, 0.05, 0.2123, 3_004_198.18)

0.7994101152481897

#### In order to find $V_0$ and $\sigma_V$, we define the following functions:

$$F(V_0,\sigma_V)= V_0 N(d_1)-De^{rT}N(d_2)-E_0$$

$$G(V_0,\sigma_V)= N(d_1)\sigma_V V_0 \frac{1}{E_0}-\sigma_E$$

Our goal is to find the vector $(V_0,\sigma_V)$ such as:
$$F(V_0,\sigma_V) = 0$$
$$G(V_0,\sigma_V) = 0$$
That means we need to find the roots of these two functions. The problem is that there is no closed form solution to this equation. We must vary the $(V_0,\sigma_V)$ vector with a trial and error process in order to find the roots. Fortunately, many multi-objective optimisation programms already exist in the scipy.optimize library.

In [58]:
def fonction_a_optimiser_1(X):
    
    V0 = X[0]
    sigma_asset = X[1]
    
    E0 = 3_004_198.18 #in million
    D = 10_000_000   #in million
    T = 1
    r = 0.05 
    sigma_equity = 0.7994101152481897
    
    a = BS_call(V0, D, T, r, sigma_asset)-E0
    
    b = volatility_of_equity(V0, D, T, r, sigma_asset, E0)-sigma_equity
    
    return [a, b]

Let's test the function with the right values and see if we get $[0,0]$

In [66]:
liste = fonction_a_optimiser_1([12_400_000, 0.2123])
print(liste)

[0.004797438625246286, 0.0]


We are close enough to $[0,0]$ to say that the function is well defined

Now let's find the roots. The purpose of HYBRD is to find a zero of a system of N non-linear functions in N variables by a modification of the Powell hybrid method.  The user must provide a subroutine which calculates the functions.  The Jacobian is then calculated by a for-ward-difference approximation.

In [98]:
sol_1 = scipy.optimize.root(fonction_a_optimiser_1, [1_000_000, 0.5],method='hybr')
print(sol_1.qtf)

[ 3.00419717e+06 -4.36301706e+01]


<ipython-input-2-46b24d4f2dce>:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(V0 / D) + (r + 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))
<ipython-input-2-46b24d4f2dce>:12: RuntimeWarning: invalid value encountered in log
  d2 = (np.log(V0 / D) + (r - 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))


**We get a really good approximation for the value of the company's assets $V_0$.** However the HYBRYD method do not accept the constraint on the volatility of assets: $0\leq  \sigma_V$ and the value of assets $0\leq V_0$ This is a huge problem and the reason for the error message **"invalid value encountered in log"**. How can we resolve this issue?

In [96]:
def fonction_a_optimiser_2(X):
    
    V0 = m.exp(X[0]) 
    sigma_asset = m.exp(X[1])
    
    E0 = 3_00.419818 #in million
    D = 10_00.0000   #in million
    T = 1
    r = 0.05 
    sigma_equity = 0.7994101152481897
    
    a = BS_call(V0, D, T, r, sigma_asset)-E0
    
    b = volatility_of_equity(V0, D, T, r, sigma_asset, E0)-sigma_equity
    
    return [a, b]

You may have noticed that the output of the function above isn't the list: $$[F(V_0,\sigma_V), G(V_0,\sigma_V)]$$ but rather: $$[F(e^{V_0},e^{\sigma_V}),G(e^{V_0},e^{\sigma_V})]$$

As explained above **optimization or root finding functions do not allow us to add a constraint** on $V_0$ and $\sigma_V$.

Our constraints being: $0\leq  V_0$ and $0\leq  \sigma_V$. We do not want our root finding algorithm to search out negative values

The solution we apply here is to find a **bijective function** $f$, such as $f:  [-\infty,+\infty] \longmapsto[0,+\infty]$

First, we apply $f$ to $(V_0,\sigma_V)$. Then, we minimise $[F(f(V_0),f(\sigma_V)), G(f(V_0),f(\sigma_V))]$.

The function $f(x) = e^{x}$ is a good candidate since it satisfies the condition above. It would be even smarter to find a constant $\alpha$ such as $f(x) = \alpha e^{x}$ depending of $F$ and $G$ magnitude's order.

In [97]:
sol = scipy.optimize.minimize(fonction_a_optimiser_2, x0=[1_000_000, 0.5],method='Nelder-Mead')
print(sol.qtf)
print('----------------------------------------------------------------------------------')
print('The value of company s asset is')
print(m.exp(sol.qtf[0]))
print('----------------------------------------------------------------------------------')
print('The value of asset s volatility is')
print(m.exp(sol.qtf[1]))

OverflowError: math range error

Absurd values 

**From now on, we will first run the multi-objective optimization. Since the value of the company's assets $V_0$ is really well approximated we will consider its value as found. Then we will perform a single-objective optimization in order to find the volatility of assets $\sigma_V$, where $V_0$ will be a constant. That's not the optmal solution but its works well in practice**

We found above $V_0 = 3 004 197.17$

In [85]:
def fonction_v_constante(sigma_asset):
    
    if not 0<=sigma_asset<=1 :   
        return np.nan
    
    V0 = 3_004_197.17
    E0 = 3004198.18
    D = 10000000
    T = 1
    r = 0.05 
    sigma_equity = 0.8
    
    b = volatility_of_equity(V0, D, T, r, sigma_asset, E0)-sigma_equity
    
    return b

In [86]:
fonction_v_constante(0.2123)

-0.7999999891558043

In [95]:
sol = scipy.optimize.root(fonction_v_constante, x0=0.3,method='hybr')
print(sol.qtf)

[0.79996661]


Finally we get the value of volatility of assets $$\sigma_v = 0.79056266$$

#### Now we have $(V_0, \sigma_v)$ we can calculate the risk-neutral probability that the company will default on the debt
$$1-N(d_2) = N(-d_2)$$

In [7]:
#The values we were missing and got with the optimization 
V0 = 3_000_000
sigma_asset = 0.2122

#The values we already got
D = 10_000_000
T = 1
r = 0.05 

d2 = (np.log(V0 / D) + (r - 0.5 * sigma_asset ** 2) * T) / (sigma_asset * np.sqrt(T))
print (d2)

probability_of_default = si.norm.cdf(-d2, 0.0, 1.0)
print(probability_of_default)

-5.544237626418172
0.9999999852381166
